# Webscraping Financial Data - Upwork listing

## Project Goal

The client has requested data to be scraped from various markets and placed into an excel table.  The links to all requested markets have been included as part of an excel spreadsheet.

## Preliminary Thought process

Data Collection:
* Parse all hyperlinks from excel spreadsheet

* Webscrape relevant values from hyperlink

* Create function to loop through and repeat for each link

Data Cleaning and Input:
* Take numbers scraped from web pages

* Input to relevant cells

* Format accordingly

## Import tools required

In [ ]:
import openpyxl
from bs4 import BeautifulSoup
from requests import get
import lxml

## Relevant information to be webscraped

List of things we need:
    * Profit - done
    * Balance - done
    * Equity - done (/x80 is a Euro sign)
    * Deposits - done
    * Withdrawals - done
    * Trades - done
    * Won(%) - done
    * Av. Trade Time - done
    * Profit Factor - done
    * Daily - done
    * Monthly - done
    * Trades per month - done
    * Expectancy - done
    

## Parsing hyperlinks from excel spreadsheet

In [ ]:
# Import data
wb = openpyxl.load_workbook(# path to spreadsheet)
ws = wb['Sheet1']

# Select columns
hyperlinks = ws["B"]

# Extract hyperlinks starting from row 2, column 2
hyperlinks_list= []
market_list = []
for num in range(len(hyperlinks)-1):
    link = (ws.cell(row=num+2, column=2))
    
    # Create list of market names
    market = link.value
    market_list.append(str(market))
    
    # Convert strings into hyperlinks and append to list
    hyperlink = link.hyperlink.target
    hyperlinks_list.append(hyperlink)

In [ ]:
print(market_list[0]), print(hyperlinks_list[0])

## Webscraping data from tables

In [ ]:
# Create soup object
link = hyperlinks_list[0]
url = get(link)
soup = BeautifulSoup(url.text, 'lxml')

In [ ]:
# Scrape relevant data and convert to text
table = soup.find('div', class_="tab-pane active")
table_text = table.text.strip()
print(" ".join(table_text.split()))

## Create a function to scrape for all markets

In [ ]:
def table_scraper(markets, hyperlinks):
    """
    Function used to loop over markets and hyperlinks to markets and extract the desired tables.
    """
    for m, h in zip(markets, hyperlinks):
        print(f'Getting data for {m}...')
        url = get(h)
        soup = BeautifulSoup(url.text, 'lxml')
        
        table = soup.findAll("li", {"class":"list-group-item"})
        
        # Profits
        pr = soup.findAll("div", {"class": "number"})
        pro = pr[2].text.strip()
        profit = pro.encode('ascii', errors='ignore')
        
        
        # Balance
        ba = soup.find("li", class_="list-group-item")
        bal = ba.text.strip().split()
        balance = bal[1].encode('ascii', errors='ignore')
        
        
        # Equity
        eq = table[1].text.split()
        equity = eq[2].encode('ascii', errors='ignore')
        
        
        # Deposits
        de = table[3].text.split()
        deposits = de[1].encode('ascii', errors='ignore')
        
        
        # Withdrawals
        wi=table[4].text.strip().split()
        withdrawals=wi[1].encode('ascii', errors='ignore')
        #print
        
        # Trades
        tr = table[5].text.split()
        trades = tr[1].encode('ascii', errors='ignore')
        
        
        # Won
        wo = table[7].text.strip().split()
        won = wo[1].encode('ascii', errors='ignore')
      
        
        # Average trade time
        avg_trade_time = table[8].text.strip().split()
        att = ' '.join(avg_trade_time[3:5])
          
        
        # Profit Factor
        pf = table[10].text.strip().split()
        profit_factor = pf[2].encode('ascii', errors='ignore')
        
    
        # Daily
        da = table[11].text.strip().split()
        daily = da[1].encode('ascii', errors='ignore')
        
        
        # Monthly
        mo = table[12].text.strip().split()
        monthly = mo[1].encode('ascii', errors='ignore')
        
        
        # Trades per month
        tpm_ = table[13].text.strip().split()
        tpm = tpm_[3].encode('ascii', errors='ignore')
      
        
        # Expectancy
        ex = table[14].text.strip().split()
        expectancy = ex[4].encode('ascii', errors='ignore')
        
        #print("\n")
        print(f'Writing data for {m}...')
        database = open(os.path.join(os.path.expanduser(
        '~'), 'Desktop', 'webscraped-info.txt'), 'a', newline='')
        database.write(f"{m};{profit};{balance};{equity};{deposits};{withdrawals};{trades};{won};{att};{profit_factor};{daily};{monthly};{tpm};{expectancy}\n")
        print(f"{m} completed!")
        print("\n")

    print('All completed!')
    
table_scraper(market_list, hyperlinks_list)

### Problems encountered:

* Note to self: be careful when webscraping.  Incorrect list slicing of web elements lead to job taking a lot longer than it should have.

* Loops in loops = no bueno.  Running `for x,y in zip(list_a, list_b)` was the answer.

* Financial data can include currency symbols which Jupyter doesn't like.  Solved by using `encode()`

### Closing thoughts:

* Plan did not go as thought.  Instead, I wrote to a text file which I could import into Excel because it would be significantly quicker to copy and paste instead of import it via python.

* Ended up taking the output from the text file, importing into Excel, and editing there.